In [ ]:
import os
import pickle

from dotenv import load_dotenv
import openai
from openai import OpenAI, AzureOpenAI

In [ ]:
load_dotenv()

In [ ]:
# client = OpenAI(
#   api_key = os.getenv("OPENAI_API_KEY"),
# )

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

### Add context to maintain conversation continuity.

In [ ]:
def save_messages(messages: list[dict]):
    try:
        with open('./db.dat', 'wb') as f:
            db = {'data': messages}
            pickle.dump(db, f)
    except:
        print('Failed to save messages.')


def load_messages() -> list[dict]:
    try:
        with open('./db.dat', 'rb') as f:
            db = pickle.load(f)
            return db.get('data', [])
    except:
        print('Failed to load messages.')
        return []

In [ ]:
messages = []
# messages = load_messages()

system_prompt = "請扮演一位智能助理，記住我告訴過你的資訊。"
messages.append({"role": "system", "content": system_prompt})
print(f'System: {system_prompt}')

while True:
    try:
        user_prompt = input("User: ")
        if not user_prompt.strip():
            break
        messages.append({"role": "user", "content": user_prompt})
        print(f'User: {user_prompt}')

        completion = client.chat.completions.create(
            model="gpt-35-turbo-120",
            messages=messages
        )
        assistant_prompt = completion.choices[0].message.content
        messages.append({"role": "assistant", "content": assistant_prompt})
        print(f'ChatGPT: {assistant_prompt}')

    except openai.APIError as err:
        print(f"Error: {err}")
        break

# save_messages(messages)

### Create a Chatbot with Gradio
Reference: https://www.gradio.app/guides/quickstart

In [ ]:
import gradio as gr

In [ ]:
def get_response(message, history, system_prompt, stream, temperature):
    messages = []
    messages.append({"role": "system", "content": system_prompt})
    for user_prompt, assistant_prompt in history:
        messages.append({"role": "user", "content": user_prompt})
        messages.append({"role": "assistant", "content": assistant_prompt})
    messages.append({"role": "user", "content": message})

    completion = client.chat.completions.create(
        model="gpt-35-turbo-120",
        messages=messages,
        stream=stream,
        temperature=temperature
    )

    if stream:
        response = ''
        for chunk_completion in completion:
            if chunk_completion.choices:
                response += chunk_completion.choices[0].delta.content or ''
                yield response
    else:
        response = completion.choices[0].message.content
        yield response


app = gr.ChatInterface(
    get_response,
    additional_inputs=[
        # Reference: https://www.gradio.app/guides/creating-a-chatbot-fast#additional-inputs
        gr.Textbox("You are helpful AI.", label="System Prompt"),
        gr.Checkbox(label='Stream', value=True),
        gr.Slider(0, 2, value=0.8, label="Temperature")
    ]
)


app.queue().launch(debug=True)

練習：製作一個「多國語言翻譯」的聊天機器人。

In [ ]:
def get_response(message, history, language, stream, temperature):
    completion = client.chat.completions.create(
        model="gpt-35-turbo-120",
        messages=[
            {"role": "system", "content": f'請將使用者的輸入翻譯成 {language} 且文法正確'},
            {"role": "user", "content": message},
        ],
        stream=stream,
        temperature=temperature
    )

    if stream:
        response = ''
        for chunk_completion in completion:
            if chunk_completion.choices:
                response += chunk_completion.choices[0].delta.content or ''
                yield response
    else:
        response = completion.choices[0].message.content
        yield response


app = gr.ChatInterface(
    get_response,
    additional_inputs=[
        gr.Textbox("english", label="Language"),
        gr.Checkbox(label='Stream', value=True),
        gr.Slider(0, 2, value=0.8, label="Temperature")
    ]
)


app.queue().launch(debug=True, share=True)

### Create a Chatbot with LINE in Replit

Reference: 
- https://developers.line.biz/console/
- https://github.com/line/line-bot-sdk-python/blob/master/examples/flask-echo/app_with_handler.py
- https://replit.com/

In [ ]:
import os

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import Configuration, ApiClient, MessagingApi, ReplyMessageRequest, TextMessage
from linebot.v3.webhooks import MessageEvent, TextMessageContent

from openai import OpenAI, AzureOpenAI


app = Flask(__name__)

configuration = Configuration(
    access_token=os.getenv('LINE_CHANNEL_ACCESS_TOKEN')
)
handler = WebhookHandler(os.getenv('LINE_CHANNEL_SECRET'))

# client = OpenAI(
#   api_key = os.getenv("OPENAI_API_KEY"),
# )

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)


@app.get("/")
def index():
    return 'Server is running!'


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token / channel secret.")
        abort(400)

    return 'OK'


def get_chatgpt_response(user_prompt: str) -> str:
    system_prompt = "請扮演使用者的愛人，並用少女的方式回答使用者的問題。"

    completion = client.chat.completions.create(
        model="gpt-35-turbo-120",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    response = completion.choices[0].message.content
    return response


@handler.add(MessageEvent, message=TextMessageContent)
def message_text(event):
    print('========== Received a message. ==========')
    print(f'::Event:: {event}')

    print('========== Send a message. ==========')
    print(f'::UserID:: {event.source.user_id}')
    user_prompt = event.message.text.strip()
    print(f'::User:: {user_prompt}')
    chatgpt_response = get_chatgpt_response(user_prompt)
    print(f'::ChatGPT:: {chatgpt_response}')

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=chatgpt_response)]
            )
        )


if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000, debug=True)
